# Exporting weights into different format

In [ ]:
!python export.py --weights runs/train/exp3/weights/best.pt --include pb

# Detection - you can pass: video, image or a folder

In [ ]:
!python detect.py --weights runs/train/exp3/weights/best.pt --img 640 --conf 0.3 --source C:/Users/Mateusz/OneDrive/Desktop/test_imgs

# Detection - from a webcam

In [ ]:
!python detect.py --weights runs/train/exp3/weights/best.pt --img 640 --conf 0.3 --source 0

# Move labelled images from one folder to folder compliant with YOLO folder structure

In [ ]:
def move_images_with_labels(from_folder_path = '../test_imgs',
                            to_folder_path = '../train_data/images/train',
                            labels_path = '../train_data/labels/train'):
    import os,shutil
    from_folder = os.listdir(from_folder_path)
    labels = os.listdir(labels_path)

    for el in labels:
        if f'{el[:-4]}.jpg' in from_folder:
            shutil.move(f'{from_folder_path}/{el[:-4]}.jpg',f'{to_folder_path}/{el[:-4]}.jpg')
        else:
            continue


# Set Up + train model(optional)

In [ ]:
import torch
import cv2
import numpy as np

def setup_model(clone_repo_and_install_req = False, 
                clear_pycatche = True,
                train = True):

    def clear_pychatche(list_of_directories=['.','./models','./utils']):
        import os,shutil

        for path in list_of_directories:
            for el in os.listdir(path):
                if el =='__pycache__':
                    shutil.rmtree(path+'/'+el)

    if clone_repo_and_install_req:
        !git clone https://github.com/ultralytics/yolov5

    %cd yolov5
    %pip install -qr requirements.txt psutil wandb

    import utils
    display = utils.notebook_init()


######### this part needs to be changed before custom training #########
    if train:
        !python train.py --img 640 --batch 1 --epochs 1 --data custom_classes.yaml  --weights yolov5s.pt --cache   
########################################################################

    if clear_pycatche:
        clear_pychatche()


In [ ]:
class Detector:

    def __init__(self, 
                 weights_path = 'yolov5/runs/train/exp3/weights/best.pt' ):
                 
        self.cap = cv2.VideoCapture(0)
        self.model = torch.hub.load('ultralytics/yolov5', 'custom', path = weights_path)
        device = 'cuda' if torch.cuda.is_available() else 'cpu'
        self.model.to(device)

    def calculate_boxes(self,results, 
                    frame, 
                    detection_treshold = 0.6, 
                    draw_bbox = True, 
                    draw_center = True, 
                    return_cords = True):

        labels, bbox, scores = results     
        x_shape, y_shape = frame.shape[1], frame.shape[0]   
        bbox = list(bbox)
        
        if len(scores) > 0:
                scores = list(map(float, scores.reshape(1, -1)[0]))
                
                indices = cv2.dnn.NMSBoxes(bboxes=bbox, scores=scores, score_threshold=detection_treshold, nms_threshold=0.8)

                for i in indices:
                        box = bbox[i]
                        x1, y1, w, h = int(box[0]*x_shape), int(box[1]*y_shape), int(box[2]*x_shape/2), int(box[3]*y_shape/2)

                        center = x1,y1
                        radius_w, radius_h = w,h 

                        if draw_center:
                                cv2.circle(frame, center=(x1,y1),radius=0, color=(0, 0, 255), thickness=5)
                                                       
                        if draw_bbox:
                                bgr = (0, 255, 0)
                                cv2.rectangle(frame, (x1-w, y1-h), (x1+w, y1+h), bgr, 2)

                        if return_cords:
                                return frame, center, radius_w, radius_h
                              
        return frame, 0, 0, 0,
    
    def find_bboxes(self,frame):
                
                
        frame = [frame]
        results = self.model(frame)
                
        labels, bbox, scores = results.xywhn[0][:, -1], results.xywhn[0][:, :-2], results.xywhn[0][:, -2:-1]

        #format required by cv2.dnn.NMS      
        return np.array(labels.cpu()), np.array(bbox.cpu()), np.array(scores.cpu())

    def run_detection(self):

        while True:

                success, frame = self.cap.read()
                
                if not success:
                        break
                        
                results = self.find_bboxes(frame)
                frame, center, radius_w, radius_h = self.calculate_boxes(results, frame)
                
                if radius_w>0:
                        yield center, radius_w, radius_h

                # cv2.imshow(frame,'vid')
                k = cv2.waitKey(1)
                if k == 27:
                        self.cap.release()
                        cv2.destroyAllWindows()
                        break
    

In [ ]:
d = Detector()

for el in d.run_detection():
    print(el)